## Deep Reasearch
#### classic cross business use cases!

In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
load_dotenv(override=True)

True

### OpenAI Agents SDK includes the following hosted tools:

The WebSearchTool lets an agent search the web. 

The FileSearchTool allows retrieving information from your OpenAI Vector Stores.

The ComputerTool allows automating computer use tasks like taking screenshots and clicking.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [3]:
INSTRUCTIONS = """You are a research assistant. Given a search term, you search the web for that term and 
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 
words. Capture the main points. Write succintly, no need to have complete sentences or good 
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the 
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."""

search_agent = Agent(
    name="Research Assistant",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [5]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)
display(Markdown(f"### Search Summary for: {message}\n\n{result.final_output}"))

### Search Summary for: Latest AI Agent frameworks in 2025

In 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing autonomous systems:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents and supports complex interaction logic. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes large language models and mathematical frameworks to design adaptive AI agents capable of real-time parameter reconfiguration. It defines a multidimensional parameter space to capture agent behaviors, supporting scalable and ethical AI applications. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system that introduces a standardized Agent-to-Agent communication layer, allowing independent agents to coordinate through asynchronous interactions. It incorporates a layered memory system for structured knowledge retention and continual learning. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data and enhancing agent robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **Google Antigravity**: An AI-powered integrated development environment (IDE) that enables developers to delegate complex coding tasks to autonomous AI agents. It introduces an "agent-first" paradigm, shifting from traditional AI code assistance to a system where AI agents operate with greater autonomy. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Google_Antigravity?utm_source=openai))

- **OpenAI Agents SDK**: A lightweight Python framework focusing on creating multi-agent workflows with comprehensive tracing and guardrails. It offers provider-agnostic compatibility with over 100 different large language models. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Google Agent Development Kit (ADK)**: A modular framework that integrates with the Google ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and requires minimal code for efficient development. ([jlcnews.com](https://www.jlcnews.com/post/the-best-ai-agents-in-2025-tools-frameworks-and-platforms-compared?utm_source=openai))

- **Agent 365**: Microsoft's platform designed to manage and deploy AI agents across various environments. It reflects Microsoft's vision of agents as the "apps" of the AI era, supporting those built via Copilot Studio, Microsoft Foundry, or open-source frameworks. ([windowscentral.com](https://www.windowscentral.com/microsoft/microsoft-doubles-down-on-agentic-ai-agent-365-prepares-for-a-future-with-over-1-billion-agents?utm_source=openai))

- **Foundry**: Microsoft's platform aimed at streamlining the deployment, management, and optimization of AI agents in enterprise environments. It offers features like unified monitoring and semantic visualization of deployed systems. ([itpro.com](https://www.itpro.com/technology/artificial-intelligence/microsoft-unveils-foundry-overhaul-for-managing-optimizing-ai-agents?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, designed to independently execute complex real-world tasks without continuous human supervision. It is one of the first fully autonomous AI agents capable of independent reasoning, dynamic planning, and autonomous decision-making. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

- **Kruti**: A multilingual AI agent and chatbot developed by the Indian company Ola Krutrim. It is designed to perform real-world tasks for users, such as booking taxis and ordering food, by integrating directly with various online services. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the advancement of autonomous systems across various domains. 

We will now use Structured Outputs, and include a description of the fields

In [6]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."



In [7]:
# Use pydantic to define the output schema for structured outputs

class WebSearchItem(BaseModel):
    reason: str = Field(description=" Your Reason for why this search is important to the query")
    query: str = Field(description="The search query term to use for the web search")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description=f"A list of web  search items to perform to best answer the query ")

plan_agent = Agent(
    name="Search Planner",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)  

In [8]:
message = "Latest AI Agent frameworks in 2025"
with trace("Search Planning"):
    plan_result = await Runner.run(plan_agent, message)
    print(plan_result.final_output)

searches=[WebSearchItem(reason='To gather information on the newest AI agent frameworks available in 2025, including comparisons and features.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To find expert opinions and analyses on the top AI agent frameworks released in 2025.', query='top AI agent frameworks reviews 2025'), WebSearchItem(reason='To explore the advancements and technologies influencing AI agent frameworks developed in 2025.', query='AI agent frameworks technology trends 2025')]


In [9]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("phanishankar134@gmail.com")  # Change to your verified sender
    to_email = To("pps130486@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [10]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_html_email],
    model="gpt-4o-mini",
)

In [11]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)
class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings")

    markdown_report: str = Field(description="The final report")

    followup_questions: str = Field(description="Suggested topic to research further")

writer_agent=Agent(
    name="Report Writer Agent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)
